In [1]:
import pandas as pd
from Bio import SeqIO
from io import StringIO
import requests
import esm
from trainer import reshape_features
import torch
from sklearn.metrics import classification_report
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tool import model as md
from tool.att import Attention
from tool import config as cfg
import numpy as np
from tqdm import tqdm
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
from featurizer import set_model, get_rep_seq
model,batch_converter,alphabet = set_model()

2024-02-21 01:56:23.402998: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 01:56:23.403042: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 01:56:23.403072: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 01:56:23.413635: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 01:56:24.577413: W tensorflow/c

Using device: cuda


In [2]:
data = pd.read_csv("./DATA/openset.csv")
data = data[data["seq"]!="-"]
data = data[data["comple_portal_num"].isin([1,2,3,4,5,6,7,8,10,12])]
data.comple_portal_num.value_counts()
data.seq = data.seq.apply(lambda x:x[:1000])
df_data = list(zip(data.index,data.seq))

In [3]:
# Embedding
stride = 20
num_iterations = len(df_data) // stride
if len(df_data) % stride != 0:
    num_iterations += 1
    
all_results = pd.DataFrame()
all_emb = []
for i in tqdm(range(num_iterations)):
    start = i * stride
    end = start + stride

    current_data = df_data[start:end]
    rep33 = get_rep_seq(current_data,model,batch_converter,alphabet)
    all_emb.append(rep33.values)
data['emb'] = np.concatenate(all_emb).tolist()

100%|██████████| 182/182 [08:18<00:00,  2.74s/it]


In [10]:
# data.to_feather("DATA/openset_with_feat.feather")

In [21]:
data

,entry,comple_portal_num,uniprot_num,Subunit_structure,seq,emb
1,A0A1D5P2B5,1,2,SUBUNIT: Homodimer. Heterodimer; with a rar mo...,MDTKHFLPLDFSNQVNSTSLNSPTSRGPMATPSLHPSIGPGIGSSL...,"[-0.03896661475300789, -0.01600981317460537, 0..."
2,A0A287A1S6,2,-,-,MFSFVDLRLLLLLAATALLTHGQEEGQEEGQQGQEEDIPPVTCVQN...,"[-0.01625731587409973, 0.005356477573513985, -..."
3,A0A2P9E102,2,-,-,MMPHPIQDAVLRVDRLSVVYPGGVTALRDTSIAFRRGEFTVLLGLS...,"[-0.061695389449596405, 0.015275530517101288, ..."
4,A0A5B9,1,0,Alpha-beta TR is a heterodimer composed of an ...,DLKNVFPPKVAVFEPSEAEISHTQKATLVCLATGFYPDHVELSWWV...,"[0.0363420732319355, -0.09713069349527359, -0...."
5,A0A8C0MYN9,3,-,-,MTSFVQKGTWLLLALLQPAVISAQQQAIDGGCSHLGQSYADRDVWK...,"[-0.03345946595072746, -0.01112181693315506, -..."
...,...,...,...,...,...,...
3669,Q9Z266,1,0,Component of the biogenesis of lysosome-relate...,MAAAGSAAVSGAGTPVAGPTGRDLFAEGLLEFLRPAVQQLDSHVHA...,"[-0.03985798358917236, -0.00685777748003602, 0..."
3670,Q9Z321,1,-,-,MKTVLMVAEKPSLAQSIAKILSRGNMSSHKGLNGACSVHKYTGTFA...,"[-0.03185144066810608, -0.08266500383615494, -..."
3671,Q9ZZW7,1,0,Forms a ribonucleoprotein complex composed of ...,MAFRKSNVYLSLVNSYIIDSPQPSSINYWWNMGSLLGLCLVIQIVT...,"[0.11712799966335297, -0.011873893439769745, 0..."
3672,V6CLJ5,1,0,Component of a complex consisting of at least ...,MFSWRRGDCESQKEENRSEERKGEETIRFPTRSPFHCVLFLLTDGF...,"[-0.02380398102104664, -0.06315197050571442, -..."


#### Train

In [2]:
# Load and preprocess the dataset.
dataset = pd.read_feather("DATA/openset_with_feat.feather")
dataset['label'] = LabelEncoder().fit_transform(dataset['comple_portal_num'])
train_data, vali_data = train_test_split(dataset, test_size=cfg.TRAIN_TEST_SPLIT_SIZE,stratify=dataset['label'])

X_train = reshape_features(pd.DataFrame(train_data['emb'].tolist()))
X_val = reshape_features(pd.DataFrame(vali_data['emb'].tolist()))

print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')

# Train the model.
gru_attention_model = md.GRUWithAttentionModel(input_shape=cfg.INPUT_SHAPE, num_classes=cfg.NUM_CLASSES)
gru_attention_model.compile_model()

history = gru_attention_model.train(
    X_train, 
    train_data['label'],
    X_val, 
    vali_data['label'],
    batch_size=cfg.BATCH_SIZE, 
    epochs=cfg.EPOCHS
)


X_train shape: (2902, 1, 1280)
X_val shape: (726, 1, 1280)


2024-02-21 01:56:09.611385: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43517 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:73:00.0, compute capability: 8.6
2024-02-21 01:56:10.538011: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/200


In [13]:
val_predictions = gru_attention_model.model.predict(X_val, batch_size=cfg.BATCH_SIZE)
ground_truth_labels = vali_data['label'].values
predicted_labels = np.argmax(val_predictions, axis=1)
report = pd.DataFrame(classification_report(ground_truth_labels, predicted_labels, zero_division=0, output_dict=True, target_names=[1,2,3,4,5,6,7,8,10,12])).T
report

1/1 [==============================] - 0s 28ms/step


,precision,recall,f1-score,support
1,0.852146,0.979890,0.911565,547.000000
2,0.802817,0.471074,0.593750,121.000000
3,0.636364,0.318182,0.424242,22.000000
4,0.750000,0.375000,0.500000,16.000000
5,1.000000,0.500000,0.666667,4.000000
6,1.000000,0.444444,0.615385,9.000000
7,0.000000,0.000000,0.000000,1.000000
8,1.000000,0.500000,0.666667,2.000000
10,0.000000,0.000000,0.000000,1.000000
12,0.000000,0.000000,0.000000,3.000000


#### Transfer

In [2]:
# Replace 'your_model_path.h5' with the actual path to your saved model file
loaded_model = load_model("./model/deepsub.h5",custom_objects={"Attention": Attention},compile=False)
dataset = pd.read_feather("DATA/openset_with_feat.feather")
dataset['label'] = LabelEncoder().fit_transform(dataset['comple_portal_num'])
X_test = reshape_features(pd.DataFrame(dataset['emb'].tolist()))
predicted = loaded_model.predict(X_test)
predicted_labels = np.argmax(predicted, axis=1)

2024-02-21 01:56:50.456471: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43517 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:73:00.0, compute capability: 8.6
2024-02-21 01:56:51.374365: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


114/114 [==============================] - 3s 4ms/step


In [3]:
report = pd.DataFrame(classification_report(dataset.label.values, predicted_labels, zero_division=0, output_dict=True, target_names=[1,2,3,4,5,6,7,8,10,12])).T
report

,precision,recall,f1-score,support
1,0.819362,0.253841,0.387601,2734.000000
2,0.173858,0.680464,0.276954,604.000000
3,0.250000,0.383178,0.302583,107.000000
4,0.208791,0.243590,0.224852,78.000000
5,0.200000,0.500000,0.285714,20.000000
6,0.122222,0.244444,0.162963,45.000000
7,0.000000,0.000000,0.000000,7.000000
8,0.000000,0.000000,0.000000,12.000000
10,0.666667,0.400000,0.500000,5.000000
12,0.666667,0.125000,0.210526,16.000000
